# Data-wrangling for forecasting

This notebook provides a reference to write forecasting results in a format that can be incorporated into a workflow with modelskill.

In [39]:
import pandas as pd
import modelskill as ms

In [40]:
def path_to_file(filename):
    return f"../tests/testdata/forecast_skill/{filename}"

def melt_df_by_date(df, name):
    return pd.melt(df.reset_index(), id_vars=["date"], var_name="lead_time", value_name=name)


def window_to_horizon_representation(forecasts):
    def get_horizon_series(row):
        new_index = (pd.to_datetime(row.name) - pd.to_datetime(row.index)).days
        return pd.Series(row.values, index=new_index)

    predictions_by_horizon = forecasts.T.apply(
        get_horizon_series, axis=1
    ).dropna(axis=1, how="all")
    predictions_by_horizon.index.name = "date"

    return predictions_by_horizon

Let´s assume we have a model to forecast the next 7 days of an arbitrary variable. In such case, we can arrange the results of such model as the following dataframe.

Every row in this dataframe represents a week of forecasted values made the date denoted by the row index. Notice that every row contains only 7 values accounting for the next 7 days: Let´s call this the _window representation_.

In [48]:
forecast_model_1 = pd.read_csv(path_to_file("forecast_model_1.csv"), parse_dates=True, index_col=0)
forecast_model_1.head(10).round(2)

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10,...,2024-01-12,2024-01-13,2024-01-14,2024-01-15,2024-01-16,2024-01-17,2024-01-18,2024-01-19,2024-01-20,2024-01-21
2022-12-31,15521.82,14296.53,13821.23,13312.48,13241.17,12847.62,13003.76,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,13195.95,13037.03,12753.71,12843.03,12563.93,12801.62,13731.58,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,NaN,NaN,13207.27,12875.01,12929.46,12625.52,12845.50,13762.85,13145.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03,NaN,NaN,NaN,13342.34,13262.45,12862.78,13014.56,13883.31,13231.08,13000.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,NaN,NaN,NaN,13998.87,13387.50,13388.44,14149.71,13420.90,13135.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05,NaN,NaN,NaN,NaN,NaN,13267.53,13302.96,14088.80,13377.50,13104.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06,NaN,NaN,NaN,NaN,NaN,NaN,13804.10,14445.89,13631.93,13286.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14813.45,13893.84,13472.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14173.23,13671.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13939.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


When evaluating forecasting results, it is interesting to compare the forecast results at different horizons. We can more efficiently write the previous dataframe such that it is sorted by how long in advance were we forecasting each value. We can call this the _horizon representation_.

In [44]:
results_model_1 = window_to_horizon_representation(forecast_model_1).dropna()
results_model_1.head(10).round(2)

,1,2,3,4,5,6,7
date,,,,,,,
2023-01-07,13804.10,13302.96,13388.44,13014.56,12845.50,12801.62,13003.76
2023-01-08,14813.45,14445.89,14088.80,14149.71,13883.31,13762.85,13731.58
2023-01-09,14173.23,13893.84,13631.93,13377.50,13420.90,13231.08,13145.25
2023-01-10,13939.92,13671.91,13472.83,13286.21,13104.92,13135.85,13000.59
2023-01-11,14669.68,13872.50,13681.53,13539.68,13406.71,13277.53,13299.57
2023-01-12,13441.08,14314.18,13746.16,13610.09,13509.02,13414.27,13322.23
2023-01-13,14470.25,13174.05,13796.16,13391.43,13294.47,13222.46,13154.95
2023-01-14,13611.65,14044.00,13120.41,13563.69,13275.30,13206.22,13154.91
2023-01-15,15539.59,14242.69,14550.75,13892.66,14208.51,14003.03,13953.80


Finally, in order to compare these results with results from a different model using __modelskill__, we need to write the _horizon representation_ in long format as follows. Notice that the we have kept a `lead_time` column, which will be used later.

In [49]:
melted_results_model_1 = melt_df_by_date(results_model_1, name="model_1")
observations = pd.read_csv(path_to_file("observations.csv"), parse_dates=True)
melted_results_model_1 = pd.merge(left=melted_results_model_1, right=observations, on="date")
melted_results_model_1.head().round(2)

,date,lead_time,model_1,observation
0,2023-01-07,1,13804.10,14319.96
1,2023-01-08,1,14813.45,15205.57
2,2023-01-09,1,14173.23,14549.37
3,2023-01-10,1,13939.92,15058.72
4,2023-01-11,1,14669.68,13444.34


Now, let´s load results computed with a different model and repeat the steps above.

In [32]:
forecast_model_2 = pd.read_csv(path_to_file("forecast_model_2.csv"), parse_dates=True, index_col=0)
results_model_2 = window_to_horizon_representation(forecast_model_2).dropna()
melted_results_model_2 = melt_df_by_date(results_model_2, name="model_2")
melted_results_model_2 = pd.merge(left=melted_results_model_2, right=observations, on="date")

Finally, we need to combine the results of the two models into a single _dataframe_ object. Let's call this format, the _matched representation_.

In [54]:
matched_model_results = melted_results_model_1.merge(
    melted_results_model_2,
    how="inner",
    on=["date","lead_time", "observation"]
    ).sort_values(
        by=["date", "lead_time"]
        ).set_index(
            "date"
            )

matched_model_results = matched_model_results.reindex(
                sorted(matched_model_results.columns),
                axis=1
                )

# We save the results as well for later use
matched_model_results.to_csv(path_to_file("matched_model_results.csv"))
matched_model_results.head().round(2)

,lead_time,model_1,model_2,observation
date,,,,
2023-01-23,1,14167.08,13198.01,14513.21
2023-01-23,2,13850.32,13096.03,14513.21
2023-01-23,3,13683.14,13844.50,14513.21
2023-01-23,4,13239.73,13528.56,14513.21
2023-01-23,5,13088.73,13009.16,14513.21


The following image shows an schematic representation of the previous workflow, where the labels, _A_, _B_ and _C_ denote the _window representation_, the _horizon representation_ and the _matched representation_ respectively. Notice that, for the sake of brevity, the image shows an example where the horizon is 3 time-steps, instead of 7.

![Data wrangling](../images/forecast_data_wrangling_wbackground.png)

## Using modelskill

Departing from the _matched representation_, it is straightforward to create a _Comparer_ object using __modelskill__. After introducing the results labelled by lead_time, we can easily compare the performance at different horizons.

In [58]:
cmp = ms.from_matched(matched_model_results, mod_items=["model_1", "model_2"], aux_items=["lead_time"], obs_item="observation")
cmp.skill(by=["model", "lead_time"]).sort_index().round(2).style()